In [34]:
#Importing Modules
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import tensorflow as tf

In [35]:
#Importing dataset.
songdf= pd.read_csv("/kaggle/input/dataset-of-songs-in-spotify/genres_v2.csv", encoding='utf-8', quotechar='"')
songdf.head(10)

In [36]:
song_name = songdf["song_name"]

In [37]:
print(song_name.shape)
print(song_name.isnull().values.any())

In [38]:
song_name = song_name.values.reshape(-1,1)

In [39]:
song_name.shape

#### REPLACING NA VALUES

In [40]:
from sklearn.impute import SimpleImputer
imr = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imr = imr.fit(song_name)
imputed_data = imr.transform(song_name)
song_name = pd.DataFrame(imputed_data)
song_name = song_name.rename(columns={0:"Song-Names"})
song_name.head(3)

#### CREATING CORE
Let's create a DataFrame that contains genre, mode, and duration_ms info of the given song names. <br>
Let's name it as core.

In [41]:
core = songdf[["genre","mode","duration_ms"]]
print(core.dtypes)
print(core.head())

In [42]:
core.dtypes

Hmm, looks like dtype of genre is object, and it contains string values. Let's encode them and make convinient for M.L. algorithm. 

In [43]:
core["genre"].value_counts()

In [44]:
core = core.replace({"genre":{"Underground Rap":0, "Dark Trap":1, "Hiphop":2, "trance":3, "trap":4, "techhouse":5, "dnb":6, "psytrance": 7, "techno":8, "hardstyle":9, "RnB":10, "Trap Metal":11, "Rap":12, "Emo":13, "Pop":14}})

#### APPLYING NMF
Non-negative matrix factorization (NMF), also non-negative matrix approximation is a group of algorithms in multivariate analysis and linear algebra where a matrix V is factorized into (usually) two matrices W and H, with the property that all three matrices have no negative elements. This non-negativity makes the resulting matrices easier to inspect.

In [45]:
from sklearn.decomposition import NMF
nmf = NMF(n_components = 6)
nmf_features = nmf.fit_transform(core)

In [46]:
from sklearn.preprocessing import normalize
norm_features = normalize(nmf_features)

In [47]:
df = pd.DataFrame(norm_features)
x = df.join(song_name)
df = pd.pivot_table(x, x[[0,1,2,3,4,5]],["Song-Names"])#for indexing song_name to our df
def your_song(value):
    print("Top 5 recommendations for given music are:")
    value = df.loc[value]
    similarities = df.dot(value)
    print(format(similarities.nlargest()))

In [48]:
your_song("rap music")